In [1]:
from dnd.ml_model1 import *
from sklearn.preprocessing import OneHotEncoder, StandardScaler
print("Done here")
4

Done here


4

NOte - it could be worth taking initiative handling into a separate function - it's a simpler function?
But there may be interplay, so maybe not

In [2]:
gen = BattleDataGeneration()
cols_x, cols_y = gen.get_headings()
res =  gen.run_battle_data_1(10)
#the run_battle_data_1 is yielding x values and y values,and I need to decouple them...


In [ ]:
#get data returns the scaled x data and y data for 100000 simulated combats
x_train, y_train, scaler = get_data(100000)

In [3]:
#Need to get a pair of characters
from dnd.util import *
from dnd.combat import *

def gettestchars():
        ch1 = HumanCharacter(gen_character(), "dwarvish")
        sword1 = MediumLongsword35()
        ch1.add_weapon_to_inventory(sword1)
        ch2 = HalfOrcCharacter(gen_character())
        sword2 = MediumLongsword35()
        ch2.add_weapon_to_inventory(sword2)
        return ch1,ch2

def test_a_model(amodel, scaler):
    
    ch1,ch2 = gettestchars()
    results = get_win_probability_from_simulation(ch1, ch2,1000)
    print("get_win_probability_from_simulation", results)

    mytest=get_datum(ch1, ch2, scaler)
    mytestopposite=get_datum(ch2, ch1, scaler)

    pred1 = amodel.predict(mytest)
    pred2 = amodel.predict(mytestopposite)
    res=(pred1, pred2 )
    #Something wrong with model- it is generating a higher than 1.0 prob
    
    print("unnormalized model results", res)

    #res_norm = (res[0]/sum(res), res[1]/sum(res))

    #print("normalized model results", res_norm)
    return res

0    0.855
1    0.145
dtype: float64
0    0.16
1    0.84
dtype: float64


In [44]:
ch1, ch2 = gettestchars()
results = get_win_probability_from_simulation(ch1, ch2, 1000)
results2 = get_win_probability_from_simulation(ch2, ch1, 1000)
d1 = get_datum(ch1, ch2, scaler)
d2 = get_datum(ch2, ch1, scaler)
pred1 = model1.predict(d1)
pred2 = model1.predict(d2)
pred21 = model2.predict(d1)
pred22 = model2.predict(d2)
print(results[0], results[1], "sim 1")
print(results2[1], results2[0], "sim opposite") 

print(pred1[0][0], pred2[0][0], pred1[0][0] + pred2[0][0], "model1 and opposite")
print(pred21[0][0],pred21[0][1], pred21[0][0] + pred21[0][1], "pred model2")
print(pred22[0][1], pred22[0][0], pred22[0][1] + pred22[0][0], "pred model2 opposite")

1.0 0.0 sim 1
1.0 0.0 sim opposite
0.99432546 0.011380941 1.0057064 model1 and opposite
0.9951904 0.004809602 1.0 pred model2
0.9930143 0.006985728 1.0 pred model2 opposite


In [46]:
print(ch1)
print(ch2)

****************************
Strength: 16
Dexterity: 12
Constitution: 19
Intelligence: 10
Wisdom: 15
Charisma: 14
Effective hit points: 44

****************************
Strength: 7
Dexterity: 8
Constitution: 15
Intelligence: 14
Wisdom: 14
Charisma: 15
Effective hit points: 42



In [47]:
x = get_win_probability_from_simulation(ch1, ch2, 100000)

In [48]:
print(x)

0    0.99997
1    0.00003
dtype: float64


In [34]:
print(results[0], results[1], "sim 1")
print(results2[1], results2[0], "sim opposite") 

print(pred1[0][0], pred2[0][0], "model1 and opposite")
print(pred21[0][0],pred21[0][1], "pred model2")
print(pred22[0][1], pred22[0][0], "pred model2 opposite")

0.855 0.145 sim 1
0.84 0.16 sim opposite
0.8627044 0.13121939 model1 and opposite
0.89151096 0.10848911 pred model2
0.85399324 0.14600682 pred model2 opposite


NameError: name 'model1' is not defined

In [7]:
y_train.head()

,char1_won,char1_hp_left,char2_won,char2_hp_left
0,True,14,False,0
1,True,1,False,0
2,False,0,True,6
3,False,0,True,26
4,False,0,True,47


In [8]:
model1=build_model_1()
y_training_values = y_train[["char1_won"]].to_numpy().astype(float)
model1.fit(x_train, y_training_values, epochs=10)


Epoch 1/10
3125/3125 [==============================] - 4s 1ms/step - loss: 0.3964 - accuracy: 0.8308
Epoch 2/10
3125/3125 [==============================] - 3s 802us/step - loss: 0.3525 - accuracy: 0.8415
Epoch 3/10
3125/3125 [==============================] - 3s 807us/step - loss: 0.3465 - accuracy: 0.8432
Epoch 4/10
3125/3125 [==============================] - 3s 803us/step - loss: 0.3430 - accuracy: 0.8441
Epoch 5/10
3125/3125 [==============================] - 2s 792us/step - loss: 0.3401 - accuracy: 0.8451
Epoch 6/10
3125/3125 [==============================] - 3s 810us/step - loss: 0.3382 - accuracy: 0.8458
Epoch 7/10
3125/3125 [==============================] - 3s 822us/step - loss: 0.3369 - accuracy: 0.84630s - loss: 0.3366 - accuracy:  - ETA: 0s - loss: 0.3368 - accuracy:  - ETA: 0s - loss: 0.3367 - accuracy: 0.84
Epoch 8/10
3125/3125 [==============================] - 2s 789us/step - loss: 0.3362 - accuracy: 0.84621s - loss: 0.3378 - accu - ETA: 1s - loss: - ETA: 0s - loss: 

TypeError: test_a_model() missing 1 required positional argument: 'scaler'

In [9]:
test_a_model(model1, scaler)

get_win_probability_from_simulation 0    0.242
1    0.758
dtype: float64
unnormalized model results (array([[0.31020218]], dtype=float32), array([[0.68521714]], dtype=float32))


(array([[0.31020218]], dtype=float32), array([[0.68521714]], dtype=float32))

In [76]:
test_a_model(model1)

get_win_probability_from_simulation 0    0.92
1    0.08
dtype: float64
unnormalized model results (array([[0.94630986]], dtype=float32), array([[0.0544098]], dtype=float32))


(array([[0.94630986]], dtype=float32), array([[0.0544098]], dtype=float32))

In [14]:

model2=build_model_2()
y_training_values = y_train[["char1_won", "char2_won"]].to_numpy().astype(float)
model2.fit(x_train, y_training_values, epochs=10)


Epoch 1/10
3125/3125 [==============================] - 3s 917us/step - loss: 0.4140 - accuracy: 0.8196
Epoch 2/10
3125/3125 [==============================] - 3s 881us/step - loss: 0.3510 - accuracy: 0.8421
Epoch 3/10
3125/3125 [==============================] - 3s 986us/step - loss: 0.3431 - accuracy: 0.8444
Epoch 4/10
3125/3125 [==============================] - 3s 873us/step - loss: 0.3394 - accuracy: 0.8443
Epoch 5/10
3125/3125 [==============================] - 3s 870us/step - loss: 0.3374 - accuracy: 0.8457
Epoch 6/10
3125/3125 [==============================] - 3s 929us/step - loss: 0.3363 - accuracy: 0.8459
Epoch 7/10
3125/3125 [==============================] - 3s 851us/step - loss: 0.3359 - accuracy: 0.8459
Epoch 8/10
3125/3125 [==============================] - 3s 894us/step - loss: 0.3354 - accuracy: 0.8459
Epoch 9/10
3125/3125 [==============================] - 3s 923us/step - loss: 0.3353 - accuracy: 0.8461
Epoch 10/10
3125/3125 [==============================] - 3s 995u

In [74]:
test_a_model(model2)


get_win_probability_from_simulation 0    0.43
1    0.57
dtype: float64
unnormalized model results (array([[0.48018116, 0.5198188 ]], dtype=float32), array([[0.43092483, 0.5690752 ]], dtype=float32))


(array([[0.48018116, 0.5198188 ]], dtype=float32),
 array([[0.43092483, 0.5690752 ]], dtype=float32))

In [69]:
test_a_model(model1)

get_win_probability_from_simulation 0    0.619
1    0.381
dtype: float64
unnormalized model results (array([[0.8022876]], dtype=float32), array([[0.31684875]], dtype=float32))


(array([[0.8022876]], dtype=float32), array([[0.31684875]], dtype=float32))

In [ ]:
show_single_random_fight_and_compare_percentage_chance_vs_neural_net(model2, scaler)

In [58]:
#Adding an extra column to the y_train - will need to get it added throughout...and
#need a softmax in the model
sum([0.33931077, 0.6606893 ])

1.00000007

In [14]:
#get_datum should return the X variables for a particular pair of characters so I can use it to predict


get_datum is used by the function show_single_random_fight_and_compare_percentage_chance_vs_neural_net


Result from simulation 0.2524, then 0.2491	Neural result: 0.4713372588157654	Difference: -0.21893725881576537


{'simulation': 0.2524, 'model': 0.47133726}

In [23]:
#model.predict() is meant for batches, instead need model(x1) - but that didn't work! Not sure what needs to be prepared for model
x1 = x_train_scaled.iloc[0:1,:]
model.predict(x1)

x_train_scaled.shape

(1000, 40)

In [50]:
#Need to get a pair of characters
from dnd.util import *
from dnd.combat import *
def test_a_model(amodel, scaler):
    ch1 = HumanCharacter(gen_character(), "dwarvish")
    sword1 = MediumLongsword35()
    ch1.add_weapon_to_inventory(sword1)
    ch2 = HalfOrcCharacter(gen_character())
    sword2 = MediumLongsword35()
    ch2.add_weapon_to_inventory(sword2)

    results = get_win_probability_from_simulation(ch1, ch2,1000)
    print("get_win_probability_from_simulation", results)

    mytest=get_datum(ch1, ch2, scaler)
    mytestopposite=get_datum(ch2, ch1, scaler)

    res=(amodel.predict(mytest), amodel.predict(mytestopposite))
    #Something wrong with model- it is generating a higher than 1.0 prob
    
    print("unnormalized model results", res)
    #res_norm = (res[0]/sum(res), res[1]/sum(res))

    #print("normalized model results", res_norm)
    return res

(0    0.271
1    0.729
dtype: float64, 0    0.735
1    0.265
dtype: float64)
(1.0, 1.0)
(0.5, 0.5)


In [32]:
model.fit(x_train_scaled, (y_train[["char1_won"]]).to_numpy(), epochs=10)

Epoch 1/10
3125/3125 [==============================] - 3s 937us/step - loss: 0.4867 - accuracy: 0.7799
Epoch 2/10
3125/3125 [==============================] - 3s 1ms/step - loss: 0.3694 - accuracy: 0.8412
Epoch 3/10
3125/3125 [==============================] - 3s 923us/step - loss: 0.3528 - accuracy: 0.8421
Epoch 4/10
3125/3125 [==============================] - 3s 899us/step - loss: 0.3469 - accuracy: 0.8430
Epoch 5/10
3125/3125 [==============================] - 3s 896us/step - loss: 0.3433 - accuracy: 0.84401s - loss: 0.3461 - accuracy:  - ETA: 1s - loss: 0.3459  - ETA: 0s - los
Epoch 6/10
3125/3125 [==============================] - 3s 864us/step - loss: 0.3412 - accuracy: 0.8447
Epoch 7/10
3125/3125 [==============================] - 3s 845us/step - loss: 0.3399 - accuracy: 0.8448
Epoch 8/10
3125/3125 [==============================] - 3s 964us/step - loss: 0.3390 - accuracy: 0.84501s - loss: 0.3390 -  - ETA: 0s - los
Epoch 9/10
3125/3125 [==============================] - 3s 987

In [ ]:
print("hello")



In [35]:
model.evaluate(x_test_scaled, (y_test[["char1_won"]]).to_numpy(), verbose=2)

NameError: name 'x_test_scaled' is not defined

How do I get the values for a particular output, or for an inner portion of the network?


In [ ]:
help(model.evaluate)

## Next steps
### Generate a new pair of characters and simulate combat, then compare the win/loss ratio to the one predicted


In [20]:
def test():
    char1 = generate_character()
    char2 = generate_character()
    print(char1, char2)
    
test()

NameError: name 'generate_character' is not defined

In [35]:
dta = [4, 1, 2, 4]
cols = ["greeting", "first", "second", "third"]
import pandas as pd
res = pd.DataFrame(data=[dta], columns=cols)
print(res)

   greeting  first  second  third
0         4      1       2      4


In [51]:
from sklearn.preprocessing import OneHotEncoder
race=OneHotEncoder(categories=[["Human", "HalfOrc"]], sparse=False)

df = pd.DataFrame(data=[["Human"], ["HalfOrc"], ["Human"]], columns=["Race"])
print(df)
prefix="char1"
res=pd.DataFrame(race.fit_transform(df), columns=[f"{prefix}_is_human", f"{prefix}_is_halforc"])
print(res)

      Race
0    Human
1  HalfOrc
2    Human
   char1_is_human  char1_is_halforc
0             1.0               0.0
1             0.0               1.0
2             1.0               0.0


In [55]:
weapontypes=[w.weapon_type for w in BattleDataGeneration().weapons] #Yuck
print(weapontypes)

['Longsword', 'Warhammer', 'Falchion', 'BastardSword', 'Scimitar']
